In [1]:
# Added libraries.
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.window import Window
import pandas as pd
import numpy as np

pd.set_option('display.max_columns', 60)

spark = SparkSession.builder.master("local[*]")\
.config("spark.executor.memory", "10G")\
.config("spark.driver.memory", "10G")\
.config("spark.memory.fraction", ".8")\
.getOrCreate()

In [5]:
defun = spark.read.csv('/home/jovyan/work/datos/defun_2016.csv', header =True) 
causas = spark.read.csv('/home/jovyan/work/datos/decatcausa.csv', header =True) 
estados = spark.read.csv('/home/jovyan/work/datos/decateml.CSV', header =True) 
genero = spark.read.csv('/home/jovyan/work/datos/desexo.csv', header =True) 
edad = spark.read.csv('/home/jovyan/work/datos/deedad.csv', header =True) 
escolaridad = spark.read.csv('/home/jovyan/work/datos/deesco.csv', header =True) 

In [6]:
for colum in defun.columns:
    defun = defun.withColumn(colum, regexp_replace(colum, '\t', ''))
    
numerics = ['anio_ocur','horas','anio_nacim']

for colum in numerics:
    defun = defun.withColumn(colum, col(colum).cast("double"))

In [44]:
estados = estados.withColumn('cve_ent', regexp_replace('cve_ent', '\t', ''))\
.withColumn('cve_mun', regexp_replace('cve_mun', '\t', '')).filter('cve_mun == 000')

In [45]:
estados.toPandas()

,cve_ent,cve_mun,cve_loc,nom_loc
0,01,000,0000\t,Aguascalientes
1,02,000,0000\t,Baja California
2,03,000,0000\t,Baja California Sur
3,04,000,0000\t,Campeche
4,05,000,0000\t,Coahuila de Zaragoza
5,06,000,0000\t,Colima
6,07,000,0000\t,Chiapas
7,08,000,0000\t,Chihuahua
8,09,000,0000\t,Ciudad de M�xico
9,10,000,0000\t,Durango


In [9]:
defun_base = defun.alias('a').join(escolaridad.alias('b'), col('a.escolarida') == col('b.CVE'))\
.withColumn('edad', defun.anio_ocur - defun.anio_nacim)\
.filter('edad < 300').filter('edad > 0').filter('horas < 24').filter('escolarida < 24')\
.withColumn('id', monotonically_increasing_id())\
.withColumn("anios_edu", col("anios_edu").cast("double"))\
.drop('escolarida')

In [10]:
defun_base.limit(10).toPandas()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,causa_def,lista_mex,sexo,edad,dia_ocurr,mes_ocurr,anio_ocur,dia_regis,mes_regis,anio_regis,dia_nacim,mes_nacim,anio_nacim,ocupacion,edo_civil,presunto,ocurr_trab,lugar_ocur,necropsia,asist_medi,...,nacionalid,derechohab,embarazo,rel_emba,horas,minutos,capitulo,grupo,lista1,gr_lismex,vio_fami,area_ur,edad_agru,complicaro,dia_cert,mes_cert,anio_cert,maternas,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax,CVE,DESCRIP,anios_edu,id
0,01,003,01,001,15,0001,01,001,15,0001,C539,12A,2,55.0,31,12,1993.0,26,10,2016,3,5,1938.0,11,3,8,8,88,9,1,...,1,99,8,8,20.0,30,2,8,037,12,8,1,16,8,99,10,2016,None,9,2,88,88,888,8888,0,999,1,Sin escolaridad,0.0,0
1,01,001,01,006,8,0001,01,001,15,0001,I259,28Z,2,75.0,7,5,2010.0,12,2,2016,7,4,1935.0,11,5,8,8,88,2,1,...,1,3,8,8,15.0,19,9,4,067,28,8,1,20,8,99,2,2016,None,9,2,88,88,888,8888,0,999,3,Primaria incompleta,5.0,1
2,01,001,01,010,1,7777,01,010,1,7777,K729,35M,2,70.0,3,11,2010.0,30,8,2016,16,6,1940.0,11,5,8,8,88,9,1,...,1,1,8,8,9.0,0,11,8,080,35,8,2,19,8,99,8,2016,None,9,2,88,88,888,8888,0,999,3,Primaria incompleta,5.0,2
3,01,001,01,001,1,0444,01,001,1,7777,E112,20D,2,54.0,17,11,2007.0,9,9,2016,2,8,1953.0,11,5,8,8,88,2,1,...,1,1,9,9,5.0,0,4,2,052,20,8,2,15,9,99,9,2016,None,9,2,88,88,888,8888,0,999,1,Sin escolaridad,0.0,3
4,01,006,01,006,8,0001,01,006,8,0001,X590,51Z,2,81.0,30,12,2015.0,8,1,2016,23,11,1934.0,11,5,1,2,9,9,1,...,1,2,8,8,17.0,30,20,25,103,E51,8,1,21,8,30,12,2015,None,2,2,88,88,888,8888,0,999,9,Profesional,18.0,4
5,01,001,01,001,15,0001,01,001,15,0001,E110,20D,1,58.0,3,9,2015.0,12,1,2016,20,10,1957.0,2,5,8,8,88,2,1,...,1,1,8,8,17.0,20,4,2,052,20,8,1,16,8,3,9,2015,None,2,1,88,88,888,8888,0,999,5,Secundaria incompleta,9.5,5
6,01,001,01,001,15,0001,01,001,15,0001,E102,20D,2,37.0,24,12,2015.0,6,1,2016,14,5,1978.0,11,4,8,8,88,9,1,...,1,2,9,9,6.0,30,4,2,052,20,8,1,12,9,24,12,2015,None,2,2,88,88,888,8888,0,999,4,Primaria completa,8.0,6
7,01,001,01,001,15,0001,01,001,15,0001,I110,27A,2,68.0,11,12,2015.0,12,1,2016,19,12,1947.0,11,3,8,8,88,9,1,...,1,7,8,8,23.0,2,9,3,066,27,8,1,18,8,12,12,2015,None,2,2,88,88,888,8888,0,999,4,Primaria completa,8.0,7
8,01,001,01,001,15,0001,01,001,15,0001,G710,23Z,1,12.0,18,12,2015.0,4,1,2016,24,2,2003.0,11,1,8,8,88,2,1,...,1,7,8,8,13.0,35,6,9,061,23,8,1,07,8,18,12,2015,None,2,2,88,88,888,8888,0,999,4,Primaria completa,8.0,8
9,01,001,01,001,15,0001,01,001,15,0001,C719,13A,1,55.0,27,12,2015.0,5,1,2016,14,1,1960.0,7,5,8,8,88,2,1,...,1,3,8,8,20.0,0,2,11,042,13,8,1,16,8,27,12,2015,None,2,1,88,88,888,8888,0,999,6,Secundaria completa,11.0,9


In [11]:
features = ['edad', 'anio_ocur','horas','anios_edu']

In [88]:
from pyspark.ml.clustering import KMeans
from pyspark.ml.feature import VectorAssembler, StandardScaler
scaler = StandardScaler(inputCol='grouped_features', outputCol= 'features', withMean=True)
assembler = VectorAssembler(inputCols = features, outputCol = 'grouped_features')
assembled = assembler.transform(defun_base)
output = scaler.fit(assembled).transform(assembled)

In [89]:
output.limit(10).toPandas()

,ent_regis,mun_regis,ent_resid,mun_resid,tloc_resid,loc_resid,ent_ocurr,mun_ocurr,tloc_ocurr,loc_ocurr,causa_def,lista_mex,sexo,edad,dia_ocurr,mes_ocurr,anio_ocur,dia_regis,mes_regis,anio_regis,dia_nacim,mes_nacim,anio_nacim,ocupacion,edo_civil,presunto,ocurr_trab,lugar_ocur,necropsia,asist_medi,...,embarazo,rel_emba,horas,minutos,capitulo,grupo,lista1,gr_lismex,vio_fami,area_ur,edad_agru,complicaro,dia_cert,mes_cert,anio_cert,maternas,lengua,cond_act,par_agre,ent_ocules,mun_ocules,loc_ocules,razon_m,dis_re_oax,CVE,DESCRIP,anios_edu,id,grouped_features,features
0,01,003,01,001,15,0001,01,001,15,0001,C539,12A,2,55.0,31,12,1993.0,26,10,2016,3,5,1938.0,11,3,8,8,88,9,1,...,8,8,20.0,30,2,8,037,12,8,1,16,8,99,10,2016,None,9,2,88,88,888,8888,0,999,1,Sin escolaridad,0.0,0,"[55.0, 1993.0, 20.0, 0.0]","[-0.609813461001, -61.6221514295, 1.2157537219..."
1,01,001,01,006,8,0001,01,001,15,0001,I259,28Z,2,75.0,7,5,2010.0,12,2,2016,7,4,1935.0,11,5,8,8,88,2,1,...,8,8,15.0,19,9,4,067,28,8,1,20,8,99,2,2016,None,9,2,88,88,888,8888,0,999,3,Primaria incompleta,5.0,1,"[75.0, 2010.0, 15.0, 5.0]","[0.394850845079, -16.0205680264, 0.47403294982..."
2,01,001,01,010,1,7777,01,010,1,7777,K729,35M,2,70.0,3,11,2010.0,30,8,2016,16,6,1940.0,11,5,8,8,88,9,1,...,8,8,9.0,0,11,8,080,35,8,2,19,8,99,8,2016,None,9,2,88,88,888,8888,0,999,3,Primaria incompleta,5.0,2,"[70.0, 2010.0, 9.0, 5.0]","[0.143684768559, -16.0205680264, -0.4160319767..."
3,01,001,01,001,1,0444,01,001,1,7777,E112,20D,2,54.0,17,11,2007.0,9,9,2016,2,8,1953.0,11,5,8,8,88,2,1,...,9,9,5.0,0,4,2,052,20,8,2,15,9,99,9,2016,None,9,2,88,88,888,8888,0,999,1,Sin escolaridad,0.0,3,"[54.0, 2007.0, 5.0, 0.0]","[-0.660046676305, -24.067906274, -1.0094085943..."
4,01,006,01,006,8,0001,01,006,8,0001,X590,51Z,2,81.0,30,12,2015.0,8,1,2016,23,11,1934.0,11,5,1,2,9,9,1,...,8,8,17.0,30,20,25,103,E51,8,1,21,8,30,12,2015,None,2,2,88,88,888,8888,0,999,9,Profesional,18.0,4,"[81.0, 2015.0, 17.0, 18.0]","[0.696250136903, -2.60833761367, 0.77072125866..."
5,01,001,01,001,15,0001,01,001,15,0001,E110,20D,1,58.0,3,9,2015.0,12,1,2016,20,10,1957.0,2,5,8,8,88,2,1,...,8,8,17.0,20,4,2,052,20,8,1,16,8,3,9,2015,None,2,1,88,88,888,8888,0,999,5,Secundaria incompleta,9.5,5,"[58.0, 2015.0, 17.0, 9.5]","[-0.459113815089, -2.60833761367, 0.7707212586..."
6,01,001,01,001,15,0001,01,001,15,0001,E102,20D,2,37.0,24,12,2015.0,6,1,2016,14,5,1978.0,11,4,8,8,88,9,1,...,9,9,6.0,30,4,2,052,20,8,1,12,9,24,12,2015,None,2,2,88,88,888,8888,0,999,4,Primaria completa,8.0,6,"[37.0, 2015.0, 6.0, 8.0]","[-1.51401133647, -2.60833761367, -0.8610644399..."
7,01,001,01,001,15,0001,01,001,15,0001,I110,27A,2,68.0,11,12,2015.0,12,1,2016,19,12,1947.0,11,3,8,8,88,9,1,...,8,8,23.0,2,9,3,066,27,8,1,18,8,12,12,2015,None,2,2,88,88,888,8888,0,999,4,Primaria completa,8.0,7,"[68.0, 2015.0, 23.0, 8.0]","[0.0432183379512, -2.60833761367, 1.6607861852..."
8,01,001,01,001,15,0001,01,001,15,0001,G710,23Z,1,12.0,18,12,2015.0,4,1,2016,24,2,2003.0,11,1,8,8,88,2,1,...,8,8,13.0,35,6,9,061,23,8,1,07,8,18,12,2015,None,2,2,88,88,888,8888,0,999,4,Primaria completa,8.0,8,"[12.0, 2015.0, 13.0, 8.0]","[-2.76984171907, -2.60833761367, 0.17734464098..."
9,01,001,01,001,15,0001,01,001,15,0001,C719,13A,1,55.0,27,12,2015.0,5,1,2016,14,1,1960.0,7,5,8,8,88,2,1,...,8,8,20.0,0,2,11,042,13,8,1,16,8,27,12,2015,None,2,1,88,88,888,8888,0,999,6,Secundaria completa,11.0,9,"[55.0, 2015.0, 20.0, 11.0]","[-0.609813461001, -2.60833761367, 1.2157537219..."


In [97]:
kmeans = KMeans(k=12, seed=1)
model = kmeans.fit(output)

In [98]:
centers = model.clusterCenters()

In [99]:
centers

[array([ 0.55775845,  0.07410847,  1.07313248, -0.05055592]),
 array([-1.13466127,  0.06873175,  0.95133248,  1.16793571]),
 array([ 0.71094815,  0.07410847,  0.74095025, -1.36049987]),
 array([ 0.1687676 ,  0.07349808, -0.27054785,  1.89448699]),
 array([-1.75427537,  0.06709306, -0.88216541,  0.65301468]),
 array([-0.19998481,  0.07410847, -1.11796404,  0.13509918]),
 array([ 0.65122442,  0.07410847, -0.20572162, -0.07674949]),
 array([ -0.05295989, -20.14237542,   0.03732223,  -0.22684477]),
 array([ 0.15169016, -2.91400936,  0.03623917, -0.27563378]),
 array([ 0.72762698,  0.07410847, -1.08297978, -1.01741394]),
 array([ -0.13826941, -53.92093526,   0.18212994,  -0.43065583]),
 array([-1.06941176,  0.07357477,  0.7037936 , -0.31858309])]

Encontramos 4 grupos que, como se puede observar por el valor de los centroides, se diferencian principalmente por la edad.

In [101]:
transformed = model.transform(output)

In [102]:
causas_por_grupo = transformed.alias('a').groupBy('prediction','a.causa_def').count()\
.select("*",dense_rank().over(Window.partitionBy('prediction').orderBy(desc('count'))).alias('rn'))\
.filter('rn < 6').join(causas.alias('b'), col('a.causa_def') == col('b.CVE')).drop('causa_def','CVE')

In [103]:
causas_por_grupo.toPandas()

,prediction,count,rn,DESCRIP
0,1,4305,1,"Infarto agudo del miocardio, sin otra especifi..."
1,1,2568,2,"Agresión con disparo de otras armas de fuego, ..."
2,1,1915,3,"Diabetes mellitus no insulinodependiente, con ..."
3,1,1470,4,"Diabetes mellitus no insulinodependiente, sin ..."
4,1,1321,5,Otras cirrosis del hígado y las no especificadas
5,6,15387,1,"Infarto agudo del miocardio, sin otra especifi..."
6,6,4872,2,"Diabetes mellitus no insulinodependiente, sin ..."
7,6,3742,3,"Diabetes mellitus no insulinodependiente, con ..."
8,6,2455,4,"Enfermedad pulmonar obstructiva crónica, no es..."
9,6,2354,5,"Neumonía, no especificada"


Como se puede observar en la tabla anterior, las causas de muerte varían en los diferentes grupos pues la edad esta claramente correlacionada con la causa de muerte. Por ejemplo, el grupo 2, correspondiente a los jóvenes, muestra causas relacionadas con violencia y accidentes. Mientras que en los grupos de mayor edad promedio están relacionadas con enfermedades. Por otro lado, dentro de estos grupos, las enfermedades también varían, por ejemplo, con el grupo 0, que es el de edad promedio de 85, las causas de muerte se diferencian de los otros por incluir problemas respiratorios.

In [79]:
causas_por_estado = transformed.alias('a').groupBy('prediction','a.ent_ocurr').count()\
.select("*",dense_rank().over(Window.partitionBy('prediction').orderBy(desc('count'))).alias('rn'))\
.filter('rn < 6').join(estados.alias('b'), col('a.ent_ocurr') == col('b.cve_ent')).drop('ent_ocurr','cve_ent','cve_loc','cve_mun')

In [80]:
causas_por_estado.toPandas()

,prediction,count,rn,nom_loc
0,1,22368,1,Ciudad de M�xico
1,1,19873,2,M�xico
2,1,16257,3,Veracruz de Ignacio de la Llave
3,1,14268,4,Jalisco
4,1,11541,5,Puebla
5,3,23878,1,Ciudad de M�xico
6,3,22048,2,M�xico
7,3,16184,3,Veracruz de Ignacio de la Llave
8,3,13791,4,Jalisco
9,3,10361,5,Puebla


No hay mucha variación en los estados con mayor número de decesos entre los grupos. El único diferente es Guanajuato para el grupo de los más jóvenes (grupo 2).

In [81]:
transformed.groupBy('prediction').agg((mean(transformed.sexo)-1).alias('% mujeres')).sort('prediction')\
.show()

+----------+-------------------+
|prediction|          % mujeres|
+----------+-------------------+
|         0| 0.3859192656198773|
|         1| 0.5429702458213357|
|         2| 0.2835719827890244|
|         3|0.46076988423090603|
+----------+-------------------+



Es interesante notar que en el grupo de los más jóvenes (grupo 2), donde las causas de muerte están relacionadas con violencia, únicamnete 3 de cada 10 muertes son mujeres. Esta proporción va aumentando junto con el rango de edades, que se puede explicar a que, dado que las mujeres tienen mayor esperanza de vida, la proporción de mujeres del total de la población va aumentando con la edad.

In [82]:
print("Analisis Descriptivo para años de estudio")
transformed.groupBy('prediction').agg((mean(transformed.anios_edu)).alias('Promedio'),\
                                      (stddev(transformed.anios_edu)).alias('Desviación'),\
                                     (max(transformed.anios_edu)).alias('Máximo'),\
                                     (min(transformed.anios_edu)).alias('Mínimo')).sort('prediction')\
.show()

Analisis Descriptivo para años de estudio
+----------+-----------------+-----------------+------+------+
|prediction|         Promedio|       Desviación|Máximo|Mínimo|
+----------+-----------------+-----------------+------+------+
|         0|9.404432229577495| 4.89099458568814|  20.0|   0.0|
|         1|4.712148338231857|4.556372510642195|  20.0|   0.0|
|         2|9.684637088241518|4.626070383856325|  20.0|   0.0|
|         3|6.819831877988068|5.024800644458311|  20.0|   0.0|
+----------+-----------------+-----------------+------+------+



El promedio de años de educación va disminuyendo con la edad, probablemente porque el promedio de anios de escolaridad ha aumentado con el paso de los anios.

In [105]:
print("Analisis Descriptivo para Horas de Defunción")
transformed.groupBy('prediction').agg((mean(transformed.horas)).alias('Promedio'),\
                                      (stddev(transformed.horas)).alias('Desviación'),\
                                     (max(transformed.horas)).alias('Máximo'),\
                                     (min(transformed.horas)).alias('Mínimo')).sort('prediction')\
.show()

Analisis Descriptivo para Horas de Defunción
+----------+------------------+------------------+------+------+
|prediction|          Promedio|        Desviación|Máximo|Mínimo|
+----------+------------------+------------------+------+------+
|         0|19.066639568937664|2.5175912272588006|  23.0|  15.0|
|         1|18.201573739365394| 3.309230348839849|  23.0|  10.0|
|         2|16.799401994395655| 3.641757709756651|  23.0|  11.0|
|         3| 10.03786955354007| 5.579939929860315|  23.0|   0.0|
|         4| 5.816076717549347| 3.750975463441098|  15.0|   0.0|
|         5|4.2691060312810585| 2.990283645289818|  12.0|   0.0|
|         6|10.462322307586144|2.6296629512255834|  15.0|   0.0|
|         7| 12.05609756097561| 6.399822307301251|  23.0|   0.0|
|         8|12.046741277156023|6.6295177079481045|  23.0|   0.0|
|         9| 4.507626093853616| 2.943018251043303|  10.0|   0.0|
|        10| 13.03225806451613|6.6552555448653825|  23.0|   0.0|
|        11|16.521615926760155| 3.80283085799

No hay diferencia en esta variable.

In [104]:
print("Analisis Descriptivo para Edad")
transformed.groupBy('prediction').agg((mean(transformed.edad)).alias('Promedio'),\
                                      (stddev(transformed.edad)).alias('Desviación'),\
                                     (max(transformed.edad)).alias('Máximo'),\
                                     (min(transformed.edad)).alias('Mínimo')).sort('prediction')\
.show()

Analisis Descriptivo para Edad
+----------+------------------+------------------+------+------+
|prediction|          Promedio|        Desviación|Máximo|Mínimo|
+----------+------------------+------------------+------+------+
|         0|  78.0907357033424| 9.513833531389281| 114.0|  58.0|
|         1|  44.4302406251141|13.878376423136457|  75.0|  12.0|
|         2| 81.29242787144445|10.159408743296051| 114.0|  58.0|
|         3| 70.52903745635601|12.406904631990766| 112.0|  35.0|
|         4| 32.35341024419544|11.072045792552505|  53.0|   1.0|
|         5| 63.29388763927395|  9.62452973469887|  98.0|  34.0|
|         6| 80.19520863044579| 9.294342685810577| 114.0|  60.0|
|         7| 66.08536585365853|19.476022007584273| 104.0|   8.0|
|         8| 70.17671400357378|18.111479037084962| 110.0|   3.0|
|         9| 81.60525049735395|10.180855399097709| 114.0|  50.0|
|        10| 64.38709677419355|18.815484059484277|  97.0|   1.0|
|        11|45.693417132892534|13.743336005534621|  64.0|  

Es la principal variable que separa los datos, pues es la de mayor rango.

In [85]:
summary = model.summary

In [86]:
summary.clusterSizes

[143359, 201081, 70885, 202472]

### Análisis de resultados:

Elegimos utilizar 4 grupos porque observamos que con esta configuración se logra hacer una separación de las observaciones en grupos característicos, diferenciados principalmente por la variable 'edad' y por ende, de causas de muerte. Cuando intentamos aumentar el número de grupos identificamos que comienza a disrciminar por la variable hora y educación, que vimos no tienen efecto sobre las causas de muerte. La variable de año no aporta información para segmentar, pues más del 90% de las observaciones ocurrieron en 2015 y 2016. 